In [19]:
import joblib
import cv2
import numpy as np

import math
import utm
import matplotlib.pyplot as plt


import os


import easyocr
import os
reader = easyocr.Reader(["en",'hi'], gpu = True)

# core idea:
# pixel_dims = actual_dims * relative_ratio

def inside_aoi(target_pxl_x, target_pxl_y, H,W):
    valid = True

    h_lim_top = int(0.020*H)
    h_lim_bot = H - int(0.20*H)

    w_lim_lft = int(0.20*W)
    w_lim_rgt = W - int(0.20*W)

    h_center = H//2
    w_center = W//2

    if target_pxl_x < w_lim_lft and target_pxl_x > w_lim_rgt:
        valid = False
        return valid
    if target_pxl_y < h_lim_top and target_pxl_y > h_lim_bot:
        valid = False
        return valid
        
    w_mid_lft = w_center - int(0.20*w_center)
    w_mid_rgt = w_center + int(0.20*w_center)

    if target_pxl_x > w_mid_lft and target_pxl_x < w_mid_rgt:
        valid  = False

    return valid



def relative_ratio(actual_dims, pixel_dims):
    return pixel_dims / actual_dims

def actual_dim_from_pixel_dim(pixel_dim, relative_ratio):
    return pixel_dim / relative_ratio


def actual_depth_from_pixel_depth(pixel_depth, pixel_height, asset_name):
    ratio = relative_ratio(ACTUAL_ASSET_DIM['flower_street_pole']['height'], pixel_height)
    actual_depth = actual_dim_from_pixel_dim(pixel_depth, ratio)
    return actual_depth

# thing_classes = ['CCTV Camera_PC500', 'Traffic_Signal_TR51', 'Sign Board_SB21', 'Bollard_BO67', 'Sign_Board_SB 1001', 'Sign Board_SB170', 'Hand Pump', 'Mobile Tower_MT_1', 'Telephone Distribution Box_EB46', 'SignBoard_SB21', 'Sign Board_SB172', 'BOREWELL', 'Electric Pole_EL11', 'Street Light_LI240', 'Sign Board_SB07', 'Sign Board_SB23', 'Sign_Board_SB153', 'Traffic Signal_TR45', 'OFC Marker_01', 'Street_Light_LI2016', 'Sign_Board_SB 1004', 'Pot_TG_3', 'Street Light_LI517', 'Street_Light_LI2014_1', 'Sign Board_SB 1003', 'Traffic_Signal_Light_TR44', 'Letter Box_LT01', 'CCTV Camera_LI8001', 'Traffic Signal Light_TR12', 'Garbage_bin_GA03', 'Traffic Signal Light_TR46', 'Electric Pole_EL8006', 'SignBoard_SB09', 'Street_Light_LI649', 'Electric Distribution Box_EB500', 'Electric Pole_EL2485', 'Traffic Signal Light_TR52', 'Zebra_Crossing', 'Sign_Board_SB172', 'Electronic Display Board_DB05', 'Sign_Board_SB8001', 'Sign Board_SB193', 'Sign Board_SB190', 'Hording_5', 'Traffic Signal Overhead_TO24', 'Electric Distribution Box_EB256', 'Traffic Signal Light_TR44', 'Sign_Board_SB170', 'Manhole_MH01', 'Electric Pole_SB153', 'Electric Distribution Box_EB08', 'Electric Distribution Box_EB110', 'Bollard_BO31', 'Tree_Deciduous', 'SignBoard_SB61', 'Sign Board_SB 1001', 'Tree Guard_TG_1', 'SignBoard_SB11', 'Sign Board_SB197', 'Electric Distribution Box_EB507', 'Rectangular Manhole', 'Bollard_BO13', 'Sign_Board_SB07', 'Traffic Signal Overhead_TO34', 'Telephone Distribution Box_EB62', 'Hording_1', 'Traffic Signal Overhead_TO28', 'Street Light_LI516', 'Electric Distribution Box_EB221', 'Manhole_MH02', 'Electric Distribution Box_EB17', 'Tree Guard_TG_2', 'Electronic Display Board_DB11', 'Traffic Signal Light_TR10', 'Sign Board_SB153', 'Sign Board_SB 1007', 'Traffic_Signal_TO28', 'Sign Board_SB196', 'Traffic_Signal_TR52', 'Transformer_EL2481', 'Sign Board_SB 1004', 'Sign_Board_SB196', 'Electric Distribution Box_EB505', 'Electric Distribution Box_EB480', 'Electric Distribution Box_EB58', 'Traffic Signal Light_TR06', 'Sign Board_SB8001', 'Street_Light_LI41', 'Street Light_LI200', 'Electric Distribution Box_EB7004', 'Electric Distribution Box_EB96', 'Traffic_Signal_TR45', 'Zebra Crossing', 'Street_Light_LI5000', 'Sign Board_SB 1006', 'Sign_Board_SB197', 'Electric Distribution Box_EB38', 'Electric Distribution Box_EB455', 'Street_Light_LI240_1', 'Garbage bin_GA03', 'Street_Light_LI617', 'Street Light_LI2014_1', 'Tree Guard_TG_4', 'CCTV Camera_PC535', 'Hoarding', 'Hording_4', 'Electric Pole_EL6005', 'Street_Light_LI200', 'Electric Distribution Box_EB02', 'Garbage bin_GA04', 'Traffic_Signal_TR50', 'Mobile_Tower_MT_1']
# ACTUAL_ASSET_DIM = {  'Street_Light_LI617':10,'Sign_Board_SB172':10,  'Street Light_LI200':10,'Letter Box_LT01':5,  'SignBoard_SB11':5,'BOREWELL':10,'Sign Board_SB153':10,  'Mobile Tower_MT_1':10,'Bollard_BO13':10,  'Electric Pole_EL11':10,'Electric Pole_SB153':10,  'Electric Pole_EL6005':10,'Street Light_LI516':5,  'Street_Light_LI649':10,'Sign_Board_SB196':5,  'Street_Light_LI240_1':10,'Street_Light_LI516':10,  'Sign_Board_SB153':5,'Electric Pole_EL2485':10,  'Sign Board_SB196':5,'Sign Board_SB 1003':5,  'Electric_Pole_EL2486':10,'Hoarding':5,  'Traffic_Signal_TR52':5,'Traffic_Signal_TR45':5,  'Traffic Signal_TR45':5,'Hording_4':10,  'Mobile_Tower_MT_1':10,'Sign_Board_SB 1004':10,  'Sign_Board_SB07':5,'Sign_Board_SB170':5,  'Street_Light_LI200':5,'Street_Light_LI2016':5,  'Street_Light_LI41':5,'Traffic_Signal_TO28':5,  'Traffic_Signal_TR50':5,'Traffic_Signal_TR51':5,  'Tree_Deciduous':10,'Zebra_Crossing':5  }



kml_src = '/home/itis/Desktop/Work_Flow_OCR/CSVKML/Track_A01.csv'
lat_value = {}
lon_value = {}
center_bearing_value = {}

kml_file = open(kml_src, 'r')
line_num = 0
for line in kml_file:
    if line_num == 0:
        line_num = line_num+1
        continue
    else:
        line = (line.split(','))
        # print (line)
        lat_value[str(int(line[0])+1)] = float(line[3])
        lon_value[str(int(line[0])+1)]  = float(line[2])
        center_bearing_value[str(int(line[0])+1)] = float(line[5])

# print (lat_value, lon_value, center_bearing_value)

def CalculatelatlonfromBearing(inputLat, inputLon,  bearing, depth):
    
    depthiKM = depth/1000
    
    earthDistance = depthiKM/6371.01
    brng = math.radians(bearing)

    inputLat = math.radians(inputLat)
    inputLon = math.radians(inputLon)

    lat2 = math.asin(math.sin(inputLat)*math.cos(earthDistance) + math.cos(inputLat)*math.sin(earthDistance)*math.cos(brng))
    lon2 = inputLon + math.atan2(math.sin(brng)*math.sin(earthDistance)*math.cos(inputLat),math.cos(earthDistance)-math.sin(inputLat)*math.sin(lat2))

    lon2 = math.fmod((lon2+3*math.pi),(2*math.pi)) - math.pi

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    return lat2,lon2

def CalculateBearingfromFOV(targetPixelX, imageWidth, fov, centerBearing):
    centerPixel = imageWidth/2
    #pixelbearingRatio = 360/imageWidthPixels #1 pixel = ? heading for spherical views
    pixelbearingRatio = fov/imageWidth #for DSLRs/ Drones
    pixelDiff = targetPixelX - centerPixel
    differenceBearing = pixelDiff*pixelbearingRatio
    targetBearing =centerBearing + differenceBearing

    # print("Target Bearing: " + str(targetBearing))

    return targetBearing


def focus_pad(im, color="white"):
  H, W, C = im.shape

  h_lim_top = int(0.20*H)
  h_lim_bot = H - int(0.10*H)

  w_lim_lft = int(0.15*W)
  w_lim_rgt = W-int(0.15*W)

  h_center = H//2
  w_center = W//2

  w_mid_lft = w_center - int(0.10*w_center)
  w_mid_rgt = w_center + int(0.20*w_center)

  # margin cropped
  # im_temp = im[h_lim_top:h_lim_bot, w_lim_lft:w_lim_rgt]

  if color == "white":
    # whiten top and bottom
    im[:h_lim_top] = np.ones(im[0:h_lim_top].shape)*255
    im[h_lim_bot:] = np.ones(im[h_lim_bot:].shape)*255
    # whiten left and right
    im[:, :w_lim_lft] = np.ones(im[:, :w_lim_lft].shape)*255
    im[:, w_lim_rgt:] = np.ones(im[:, w_lim_rgt:].shape)*255
    # whiten center region
    im[:, w_mid_lft:w_mid_rgt] = np.ones(im[:, w_mid_lft:w_mid_rgt].shape)*255

  else:
    # darken top and bottom
    im[:h_lim_top] = np.zeros(im[0:h_lim_top].shape)
    im[h_lim_bot:] = np.zeros(im[h_lim_bot:].shape)
    # darken left and right
    im[:, :w_lim_lft] = np.zeros(im[:, :w_lim_lft].shape)
    im[:, w_lim_rgt:] = np.zeros(im[:, w_lim_rgt:].shape)
    # darken center region
    im[:, w_mid_lft:w_mid_rgt] = np.zeros(im[:, w_mid_lft:w_mid_rgt].shape)

  return im

def display(img):
  plt.close()
  plt.imshow(img)
  plt.xticks([], []) 
  plt.yticks([], []) 
  plt.show()


# Path to save CSV file.
csv_write = open('Vector_annotation_SOP_17_Aug.csv', 'w')
csv_write.write('id,lat,lon,Object,Score,ImageName,Output\n')
feat_id = 0

image_dir = "/home/itd/Desktop/JAIPUR_ASSETS_SEGMENTATION/segline/src/store/data/Allowed_Classes/test/"
image_dir = "/home/itis/Desktop/Work_Flow_OCR/All_SHOP_ANNOTATION/"
# Bin file Path.
bin_dir = '/home/itd/Desktop/JAIPUR_ASSETS_SEGMENTATION/segline/src/store/data/Allowed_Classes/prediction/eval_test/'
bin_dir = '/home/itis/Desktop/Work_Flow_OCR/src/store/data/Allowed_Classes/prediction/eval_All_SHOP_ANNOTATION/'
for bin_file in os.listdir(bin_dir):
    bin_file = bin_dir+ bin_file
    x = joblib.load(bin_file)
    

    # x[0].keys() --> dict_keys(['imgShape', 'predClasses', 'predBoxes', 'boxScores', 'imageName'])
    # x[1] --> thing_classes  
    thing_classes = x[1]
    imageWidth, imageHeight =x[0]['imgShape'][1], x[0]['imgShape'][0] #img.shape[1], img.shape[0]
    reference =  (int(imageHeight), int(imageWidth/2))
    image_id = x[0]['imageName'].split('.')[0].split('-')[-1]
    print (image_id)
    img = cv2.imread(image_dir+x[0]['imageName'])
    img = focus_pad(img.copy(), "black")
    # display(img)

    for num, box in enumerate(x[0]['predBoxes']):
        # print ("Class", x[0][0]['predClasses'])
        # print(x[0]['boxScoresdraw_instance_predictions'])
        # print ("Class Name", thing_classes[x[0]['predClasses'][num]])
        
        feat_id = feat_id +1
        targetPixelX =  int((box[0]+box[2])/2)
        targetPixelY = int(box[3])
        if not inside_aoi(targetPixelX, targetPixelY, imageHeight, imageWidth):
            print ("Not Inside AOI")
            continue

        aoi_img = img[int(box[1]):int(box[3]), int(box[0]):int(box[2]),:]

        output = reader.readtext(aoi_img, detail = 0, paragraph=True, contrast_ths =0.3)
        if len(output) <1:
            continue
        print (output)
        output_str = " ".join(output)

        if img[targetPixelY,targetPixelX][0] == 0 and img[targetPixelY,targetPixelX][1] == 0 and img[targetPixelY,targetPixelX][2]==0:
            # print ("Black Region")
            # display(img[targetPixelY-500:targetPixelY+500,targetPixelX-500:targetPixelX+500,:])
            continue
            
        diffX, diffY = abs(targetPixelX- reference[0]), abs(targetPixelY- reference[1])

        distance = math.sqrt(diffX**2 + diffY**2)

        print ("Distance", distance)
        if diffX < 1000:
            factor = 0.15
        if diffX < 1400:
            factor = 0.40
        else:
            factor  = 0.2 #ACTUAL_ASSET_DIM[thing_classes[x[0]['predClasses'][num]]] /10
            if targetPixelX > reference[1]:
                factor = 0.6

        depth = int(distance / 100)* factor 

        centerBearing = center_bearing_value[image_id]# scrape value from kml file
        fov = 360 # fixed
        bearing  = CalculateBearingfromFOV(targetPixelX, imageWidth, fov, centerBearing)
        inputLat , inputLon = lat_value[image_id], lon_value[image_id]# scrape value from kml file
        lat, lon = CalculatelatlonfromBearing(inputLat, inputLon,  bearing, depth)
        # content to write in csv file id,lat,lon,Object,Score,ImageName
#         print (output_str, x[0]['predClasses'][num], x[0]['imageName'])
#         print (feat_id, lat,lon, x[0]['predClasses'][num], x[0]['imageName'],output_str)
        csv_str = '{},{},{},{},{},{},{}\n'.format(feat_id, lat,lon, x[0]['predClasses'][num], str(x[0]['boxScoresdraw_instance_predictions'][num]), x[0]['imageName'],output_str)
        csv_write.write(csv_str)


csv_write.close()


3084
['अरुूणा साडी स्टोर्स']
Distance 3013.272805439295
['S.R. JEWELLERS']
Distance 3726.150426378409
['राधाराना ज्वलर्स ०']
Distance 2867.2036900087865
['ओमगणेश न्यसिल्कस्टोर७']
Distance 3361.853357896504
['जनता साड़ी हाउस']
Distance 3028.704838705812
['Silver Centrrea']
Distance 2749.098215779131
['NPY JAIN JEYELERS e']
Distance 3177.3235592240208
['रयम']
['नवरलसाई केन्द्र']
Distance 2741.207945413846
3061
['जनळलजनउ्वतर']
Distance 3645.0384085767873
['Jewellery Super mart']
Distance 2943.137951235042
['अग्रटात जॅन ज्वेल्त॰']
Distance 3465.3903099073846
['Siप', 'GANESH GENSकJEWELLERS@']
Distance 3324.8461618547103
['JOHARrS']
Distance 3754.142778318374
['५ Modi ExportHousc @']
Distance 2682.094144507236
3048
['D IDBI OANKI=']
Distance 3087.6852171165374
['रेडीमेड सेन्टर']
Distance 2978.5152341393186
['श्याम स्टेशनरी मार्ट']
Distance 3772.109887052603
Not Inside AOI
["GEM ' JEWELS PEECIoUs - SEAIPRECICUS GEADS 3lalRTH STon"]
Distance 3366.07813337718
['ईगन्नन्ड दयर -']
Distance 3822.970

['शयाम न्वैलर्स ल']
Distance 3191.214345668432
['NANIRATHUUY JEWELLERS']
Distance 3242.2697296801202
['आशका ्यलर्स ']
Distance 3584.4855976834388
[' -eश']
Distance 3384.7389855053816
['अशोकरेडीमेड हण्डीक्राफ्ट्स७']
Distance 3618.3295869779467
1379
['*', '}7']
Distance 3183.9488061211036
1385
['थाम आणरन टंडस']
Distance 2886.2129166088907
['जनरलडेन्टलक्लिट']
Distance 3034.162981779324
['श्राराधक']
Distance 3190.0710023446186
3095
['मोहनलाल  अ्रशेरकमारजल०']
Distance 3353.687522712872
['RS lextiles Variety Clofh Sicreto']
Distance 3494.3744790734722
['गर्ग ज्योल्स']
Distance 3159.637953943458
['RAJASTHANOPTICAL INDUSTRIESO']
Distance 3151.0001586797803
['आसी तास॰']
Distance 2977.165934240146
['गौर्ादास नन्दकिशर =']
Distance 2833.077831617056
['उलस२']
Distance 3555.7835985897677
['शिधाचन्द घन्यमदन']
Distance 2715.9250726041764
3059
['पनीश कुमार  ग़अग्रवालज्वैलर्स']
Distance 3838.8440968604077
['अग्रटाल जॅनज्वंर्न']
Distance 3171.2231078875543
['JOHARIS']
Distance 2715.666584836953
['मोतीरन्स

['णम']
Distance 3930.681493074706
Not Inside AOI
['शर्मा एम्पोरयिम :']
Distance 2580.104842831004
1563
1592
['जसौरिया मशीनरी स्टोर']
Distance 3462.707755500022
Not Inside AOI
['श्री गोपी पवित्र भोजनाल']
Distance 2959.3999391768593
['पनोहर सिडनगर॰']
Distance 3109.133158936748
3067
['रट्न दीप']
Distance 3278.1854737034023
['डाइंग रिटेत शोरम']
Distance 3901.4502431788105
['CafeK']
Distance 2943.4430519376456
['Iत्रीकृष्णम् =']
Distance 2615.4590419274396
['परना्मी बग्स']
Distance 3527.899374982229
['शान्तिलात दुर्लभजी @']
Distance 2764.311125759906
['गरय णालि']
Distance 3464.0809748041397
1579
['-7@', '&0 7']
Distance 3469.051887764148
['नसर्सपीराधेरयामअगत']
Distance 2683.2763555027277
1393
['ऐनन ज्चत"']
Distance 3673.7072556206763
['७ पूल एम्गाारियम']
Distance 3886.6883075440974
['रूपल्स महदीर उवतरी']
Distance 3867.185669191486
1572
['औम ऑयरन ट्रेडस']
Distance 3515.898747120002
['जनरलडन्टलक्लनक९०']
Distance 3206.3719372524456
3111
['राणा साडी सेन्ट']
Distance 2809.2490099668985
['पैरीलाल

['रजप्यन ठास्टर=६४r=i3']
Distance 3046.405915172829
['LRS न्वेलर्स =']
Distance 3820.0282721466865
1583
['चिज्ञय ँफसिश']
Distance 3380.521409487004
['सोरिया २व']
Distance 2787.1284505741746
['गोबिन्टराम रोचीयम']
Distance 3222.3421606030606
['जन उफ डिपे -']
Distance 2620.059541308174
['पकाशजुक डिणट']
Distance 3079.1675823183123
['गविन्दनक डेण -']
Distance 2695.134319472779
1365
['आसऊअग्वल']
Distance 3211.940846279707
['वचम सणन सट']
Distance 3360.1744002357973
['्दस्टडणट-']
Distance 3868.8066635591913
['यानेक स्टेशनस >']
Distance 3708.0794220188973
1567
['न्यूऔमपन्सी  जगानस्टर॰0']
Distance 3674.516838987134
['बाम्बेचँच ऐडियो नेन्टर *']
Distance 2775.3576346121595
['हितन ज्वैल']
Distance 3429.215799567009
['211 घूल एम्पोरियम', 'डेरेवाला ईन्टरप्राईजेज २१२']
Distance 3048.51439229012
['रूपल्सम*-']
Distance 3322.734566588189
['Qairtel']
Distance 3632.725285512242
['Sbades m+a"@']
Distance 3166.3734776554707
1571
['जनरलडेन्टलक्लिनिक॰']
Distance 2898.7066081271487
3062
['h IGENS&JEWELLERS@']
D

['राणासाडीएम्पेरियम']
Distance 3104.3461469365816
['RANASe']
Distance 3295.9481185237123
['रामगीपाल रामबल्लभ']
Distance 2920.651468422756
['SHRI SAI KRIPA']
Distance 2871.6993227007592
['ग्यारसानाल आमप्रकश']
Distance 2677.44822545647
['भाटिया क्लॉथ स्टोर']
Distance 3167.1250370012235
['माघालाल रामगोपालबजाज @']
Distance 3825.292799250797
['२पलक्ष्मt']
Distance 3499.946428161437
3060
['%b9e0', 'लर्स']
['NIRHILA GEMS 23633050 0']
Distance 3601.371127778974
['Sा[ GANESH GENS&JEWELLERSG']
Distance 2805.733059291279
['Jewellery Super marre']
Distance 2616.1806130311415
['JOHARIS']
Distance 3158.0006333121596
['सग्रवल जनज्वततं']
Distance 3405.125842021114
['अगठाल जन ज्वलर्स॰']
Distance 3214.508671632416
['प्तारन्सज्वलसत॰ मर्तसन्स']
Distance 3838.629182403531
1568
['हितन ज्वल']
Distance 3866.357588221762
['गल एम्पोरियम 20', 'डेरेवाला ईन्टरप्राईजेज २१२']
Distance 3436.2859310598706
['Olalrtel c']
Distance 3855.6561049969173
['रपल्स=']
Distance 3560.067415092023
1565
['0 alrtel']
Distance 3140.5

['स्ादी भण्डार 00']
Distance 3281.699712039479
['जाजाद 7-7i']
Distance 3303.5886547813425
Not Inside AOI
['RAMSNG0']
Distance 2533.686050006985
['राव् एप्ड कम्पना']
Distance 3854.8247690394437
['मतातसज्यततलि ']
Distance 2658.9370056471816
Not Inside AOI
1586
['राज स्टेशाट']
Distance 3749.360612157758
['नश १डि']
Distance 3782.662818703248
['ससय मयन']
Distance 3586.073061163144
1373
['जैन बुक डिपो']
Distance 3502.476837896291
['याससयथट']
Distance 2982.389981206348
['सोखिया भवन']
Distance 3010.5024497581794
['गौविन्दबुकडिपो']
Distance 3219.908694357652
1361
['BUNUIY शन']
Distance 3182.1517877059227
['<< स']
Distance 3624.22750389652
['श्रदा एन्टयाईजेज७']
Distance 3008.7249791232166
['मितल णतिजन स्टर॰']
Distance 3289.485370084506
['आरसा भार्गत कम्णन']
Distance 3118.3421877658006
['फप्णा स्टेशनर्नः']
Distance 2984.990787255465
1603
1377
['नेसर्स मी राधेरयामअग्रवाल']
Distance 3057.9896991324217
['चिजय चक डिणो']
Distance 3648.191058593286
['बिनदरामरोचीरम']
Distance 3848.150204968616
3099
['शि

In [20]:
!cat Vector_annotation_SOP_17_Aug.csv | wc -l

966


### 